In [186]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Dense, Input, Flatten, Dropout
from keras.optimizers import Adam, SGD
from keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy, BinaryCrossentropy
import os

dataset_path = "datasetMore/"
num_classes = 10
epochs = 200

In [93]:
def convert_img(img_path) -> np.ndarray:
    img = tf.keras.preprocessing.image.load_img(img_path, 
                                                grayscale=True, 
                                                color_mode="grayscale")
    output = tf.keras.preprocessing.image.img_to_array(img)
    output = tf.image.resize(output, size=(28,28))
    return output

In [85]:
def convert_label(label):
    output = tf.keras.utils.to_categorical(label, 
                                  num_classes)
    return output

In [162]:
def prepare_for_prediction(img_path):
    img = convert_img(img_path)
    img = tf.convert_to_tensor(img)
    
    img = tf.squeeze(img)
    
    return img

In [192]:
def load_data(dir_path): #-> tuple[tf.Tensor, tf.Tensor]:
    datas = []
    labels = []
    
    dirs = os.listdir(dir_path)
    for dir in dirs:
        
        file_path = os.path.join(dir_path, dir)
        for files_in_dir in os.listdir(file_path):
            
            img_array = convert_img(os.path.join(file_path, files_in_dir)) / 255
            datas.append(img_array)
            
            label = convert_label(dir)
            labels.append(label)

    
    return labels, datas

In [197]:
labels, datas = load_data(dataset_path)
X_train, X_test, y_train, y_test = train_test_split(labels, datas, test_size=0.4, shuffle=True, random_state=42)

X_train = tf.convert_to_tensor(X_train)
X_test = tf.convert_to_tensor(X_test)

y_train = tf.convert_to_tensor(y_train)
y_train = tf.squeeze(y_train)
y_test = tf.convert_to_tensor(y_test)
y_test = tf.squeeze(y_test)

inputLayer = Input(shape=(28,28))

output = Dense(256, activation="relu", input_shape=(28,28))(inputLayer)
output = Dropout(0.5)(output)
output = Dense(128, activation="relu")(output)
output = Dropout(0.5)(output)
output = Dense(68, activation="relu")(output)
output = Flatten()(output)
output = Dense(num_classes, activation="softmax")(output)

model = Model(inputLayer, output)
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=CategoricalCrossentropy(from_logits=True),
    metrics=["acc"]
)

model.summary()

c:\Users\Moritz\anaconda3\lib\site-packages\keras\utils\image_utils.py:409: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn(


Model: "model_33"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_36 (InputLayer)       [(None, 28, 28)]          0         
                                                                 
 dense_108 (Dense)           (None, 28, 256)           7424      
                                                                 
 dropout_15 (Dropout)        (None, 28, 256)           0         
                                                                 
 dense_109 (Dense)           (None, 28, 128)           32896     
                                                                 
 dropout_16 (Dropout)        (None, 28, 128)           0         
                                                                 
 dense_110 (Dense)           (None, 28, 68)            8772      
                                                                 
 flatten_12 (Flatten)        (None, 1904)              0  

In [198]:
model.evaluate(y_test, X_test)
model.fit(y_train, X_train, epochs=epochs)
model.save('model.keras')

c:\Users\Moritz\anaconda3\lib\site-packages\keras\backend.py:5531: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


270/270 [==============================] - 1s 3ms/step - loss: 2.3359 - acc: 0.1068
Epoch 1/200
405/405 [==============================] - 2s 4ms/step - loss: 2.2801 - acc: 0.1369
Epoch 2/200
405/405 [==============================] - 2s 5ms/step - loss: 1.7571 - acc: 0.4239
Epoch 3/200
405/405 [==============================] - 2s 4ms/step - loss: 1.4739 - acc: 0.5416
Epoch 4/200
405/405 [==============================] - 2s 4ms/step - loss: 1.2918 - acc: 0.6024
Epoch 5/200
405/405 [==============================] - 2s 4ms/step - loss: 1.1605 - acc: 0.6460
Epoch 6/200
405/405 [==============================] - 2s 4ms/step - loss: 1.0630 - acc: 0.6769
Epoch 7/200
405/405 [==============================] - 2s 5ms/step - loss: 0.9900 - acc: 0.6960
Epoch 8/200
405/405 [==============================] - 2s 5ms/step - loss: 0.9281 - acc: 0.7159
Epoch 9/200
405/405 [==============================] - 2s 4ms/step - loss: 0.9060 - acc: 0.7213
Epoch 10/200
405/405 [==============================

In [200]:
img = prepare_for_prediction("examples/Example_1.png")
input_value = tf.expand_dims(img, axis=0)

predict_value = model.predict(input_value)
print(predict_value)

1/1 [==============================] - 0s 13ms/step
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
